In [1]:
import requests
import json
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import from_utc_timestamp, to_timestamp, to_date, col
import nltk
import pandas as pd
import numpy as np
import scipy
from matplotlib import pyplot as plt
import datetime
import time
import pickle


import matplotlib.dates as mdates
import matplotlib.pyplot as plt 



Bad key "text.kerning_factor" on line 4 in
/h/224/cameron/miniconda3/envs/pyspark_env/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


In [ ]:
PUSHSHIFT_REDDIT_URL = "http://api.pushshift.io/reddit"

def fetchObjects(**kwargs):
    # Default params values
    params = {"sort_type":"created_utc","sort":"asc","size":1000}
    for key,value in kwargs.items():
        params[key] = value
    print(params)
    type = "comment"
    if 'type' in kwargs and kwargs['type'].lower() == "submission":
        type = "submission"
    r = requests.get(PUSHSHIFT_REDDIT_URL + "/" + type + "/search/",params=params)
    if r.status_code == 200:
        response = json.loads(r.text)
        data = response['data']
        sorted_data_by__id = sorted(data, key=lambda x: int(x['id'],36))
        return sorted_data_by__id

def process(**kwargs):
    max_created_utc = 1577750400
    max_id = 0
    data = pd.DataFrame()    
    #file = open("data.json","w")
    while 1:
        nothing_processed = True
        d = {'created_utc': [], 'body': [], 'subreddit': [], 'score': [],'author':[]}
        objects = fetchObjects(**kwargs,after=max_created_utc)
        if objects == None:
            print("pushshift failed")
            continue
        for object in objects:
            id = int(object['id'],36)
            if id > max_id:
                nothing_processed = False
                created_utc = object['created_utc']
                max_id = id
                if created_utc > max_created_utc: max_created_utc = created_utc
                # Code to do something with comment goes here ...
                # ...
                # insertCommentIntoDB(object)
                # print(json.dumps(object,sort_keys=True,ensure_ascii=True),file=file)
                # ...
                d['body'].append(object['body'])
                d['created_utc'].append(created_utc)
                d['author'].append(object['author'])
                d['subreddit'].append(object['subreddit'])
                d['score'].append(object['score'])
        data = data.append(pd.DataFrame(d))
        if nothing_processed: 
            data.to_parquet("dataframes/2020_comments/{subreddit}-2020_comments.parquet".format(**kwargs))
        max_created_utc -= 1
        time.sleep(1)

subreddits = ["Pete_Buttigieg"]
frames = map(lambda sub : process(subreddit=sub,type="comment",before=1588291200),subreddits)
data = pd.concat(frames)
data

{'sort_type': 'created_utc', 'sort': 'asc', 'size': 1000, 'subreddit': 'Pete_Buttigieg', 'type': 'comment', 'before': 1588291200, 'after': 1577750400}
{'sort_type': 'created_utc', 'sort': 'asc', 'size': 1000, 'subreddit': 'Pete_Buttigieg', 'type': 'comment', 'before': 1588291200, 'after': 1577753296}
{'sort_type': 'created_utc', 'sort': 'asc', 'size': 1000, 'subreddit': 'Pete_Buttigieg', 'type': 'comment', 'before': 1588291200, 'after': 1577756667}
{'sort_type': 'created_utc', 'sort': 'asc', 'size': 1000, 'subreddit': 'Pete_Buttigieg', 'type': 'comment', 'before': 1588291200, 'after': 1577760735}
{'sort_type': 'created_utc', 'sort': 'asc', 'size': 1000, 'subreddit': 'Pete_Buttigieg', 'type': 'comment', 'before': 1588291200, 'after': 1577765384}
{'sort_type': 'created_utc', 'sort': 'asc', 'size': 1000, 'subreddit': 'Pete_Buttigieg', 'type': 'comment', 'before': 1588291200, 'after': 1577768271}
{'sort_type': 'created_utc', 'sort': 'asc', 'size': 1000, 'subreddit': 'Pete_Buttigieg', 'type

{'sort_type': 'created_utc', 'sort': 'asc', 'size': 1000, 'subreddit': 'Pete_Buttigieg', 'type': 'comment', 'before': 1588291200, 'after': 1578009459}
{'sort_type': 'created_utc', 'sort': 'asc', 'size': 1000, 'subreddit': 'Pete_Buttigieg', 'type': 'comment', 'before': 1588291200, 'after': 1578012230}
{'sort_type': 'created_utc', 'sort': 'asc', 'size': 1000, 'subreddit': 'Pete_Buttigieg', 'type': 'comment', 'before': 1588291200, 'after': 1578015983}
{'sort_type': 'created_utc', 'sort': 'asc', 'size': 1000, 'subreddit': 'Pete_Buttigieg', 'type': 'comment', 'before': 1588291200, 'after': 1578020107}
{'sort_type': 'created_utc', 'sort': 'asc', 'size': 1000, 'subreddit': 'Pete_Buttigieg', 'type': 'comment', 'before': 1588291200, 'after': 1578022831}
{'sort_type': 'created_utc', 'sort': 'asc', 'size': 1000, 'subreddit': 'Pete_Buttigieg', 'type': 'comment', 'before': 1588291200, 'after': 1578027134}
{'sort_type': 'created_utc', 'sort': 'asc', 'size': 1000, 'subreddit': 'Pete_Buttigieg', 'type

{'sort_type': 'created_utc', 'sort': 'asc', 'size': 1000, 'subreddit': 'Pete_Buttigieg', 'type': 'comment', 'before': 1588291200, 'after': 1578276302}
{'sort_type': 'created_utc', 'sort': 'asc', 'size': 1000, 'subreddit': 'Pete_Buttigieg', 'type': 'comment', 'before': 1588291200, 'after': 1578281823}
{'sort_type': 'created_utc', 'sort': 'asc', 'size': 1000, 'subreddit': 'Pete_Buttigieg', 'type': 'comment', 'before': 1588291200, 'after': 1578295769}
{'sort_type': 'created_utc', 'sort': 'asc', 'size': 1000, 'subreddit': 'Pete_Buttigieg', 'type': 'comment', 'before': 1588291200, 'after': 1578307312}
{'sort_type': 'created_utc', 'sort': 'asc', 'size': 1000, 'subreddit': 'Pete_Buttigieg', 'type': 'comment', 'before': 1588291200, 'after': 1578319966}
{'sort_type': 'created_utc', 'sort': 'asc', 'size': 1000, 'subreddit': 'Pete_Buttigieg', 'type': 'comment', 'before': 1588291200, 'after': 1578323496}
{'sort_type': 'created_utc', 'sort': 'asc', 'size': 1000, 'subreddit': 'Pete_Buttigieg', 'type

{'sort_type': 'created_utc', 'sort': 'asc', 'size': 1000, 'subreddit': 'Pete_Buttigieg', 'type': 'comment', 'before': 1588291200, 'after': 1578538309}
{'sort_type': 'created_utc', 'sort': 'asc', 'size': 1000, 'subreddit': 'Pete_Buttigieg', 'type': 'comment', 'before': 1588291200, 'after': 1578541253}
{'sort_type': 'created_utc', 'sort': 'asc', 'size': 1000, 'subreddit': 'Pete_Buttigieg', 'type': 'comment', 'before': 1588291200, 'after': 1578544814}
{'sort_type': 'created_utc', 'sort': 'asc', 'size': 1000, 'subreddit': 'Pete_Buttigieg', 'type': 'comment', 'before': 1588291200, 'after': 1578551435}
{'sort_type': 'created_utc', 'sort': 'asc', 'size': 1000, 'subreddit': 'Pete_Buttigieg', 'type': 'comment', 'before': 1588291200, 'after': 1578567270}
{'sort_type': 'created_utc', 'sort': 'asc', 'size': 1000, 'subreddit': 'Pete_Buttigieg', 'type': 'comment', 'before': 1588291200, 'after': 1578576027}
{'sort_type': 'created_utc', 'sort': 'asc', 'size': 1000, 'subreddit': 'Pete_Buttigieg', 'type

{'sort_type': 'created_utc', 'sort': 'asc', 'size': 1000, 'subreddit': 'Pete_Buttigieg', 'type': 'comment', 'before': 1588291200, 'after': 1578746160}
{'sort_type': 'created_utc', 'sort': 'asc', 'size': 1000, 'subreddit': 'Pete_Buttigieg', 'type': 'comment', 'before': 1588291200, 'after': 1578755885}
{'sort_type': 'created_utc', 'sort': 'asc', 'size': 1000, 'subreddit': 'Pete_Buttigieg', 'type': 'comment', 'before': 1588291200, 'after': 1578760359}
{'sort_type': 'created_utc', 'sort': 'asc', 'size': 1000, 'subreddit': 'Pete_Buttigieg', 'type': 'comment', 'before': 1588291200, 'after': 1578763292}
{'sort_type': 'created_utc', 'sort': 'asc', 'size': 1000, 'subreddit': 'Pete_Buttigieg', 'type': 'comment', 'before': 1588291200, 'after': 1578767707}
{'sort_type': 'created_utc', 'sort': 'asc', 'size': 1000, 'subreddit': 'Pete_Buttigieg', 'type': 'comment', 'before': 1588291200, 'after': 1578770525}
{'sort_type': 'created_utc', 'sort': 'asc', 'size': 1000, 'subreddit': 'Pete_Buttigieg', 'type

...

{'sort_type': 'created_utc', 'sort': 'asc', 'size': 1000, 'subreddit': 'Pete_Buttigieg', 'type': 'comment', 'before': 1588291200, 'after': 1583282418}
{'sort_type': 'created_utc', 'sort': 'asc', 'size': 1000, 'subreddit': 'Pete_Buttigieg', 'type': 'comment', 'before': 1588291200, 'after': 1583282808}
{'sort_type': 'created_utc', 'sort': 'asc', 'size': 1000, 'subreddit': 'Pete_Buttigieg', 'type': 'comment', 'before': 1588291200, 'after': 1583283252}
{'sort_type': 'created_utc', 'sort': 'asc', 'size': 1000, 'subreddit': 'Pete_Buttigieg', 'type': 'comment', 'before': 1588291200, 'after': 1583283728}
{'sort_type': 'created_utc', 'sort': 'asc', 'size': 1000, 'subreddit': 'Pete_Buttigieg', 'type': 'comment', 'before': 1588291200, 'after': 1583284111}
{'sort_type': 'created_utc', 'sort': 'asc', 'size': 1000, 'subreddit': 'Pete_Buttigieg', 'type': 'comment', 'before': 1588291200, 'after': 1583284507}
{'sort_type': 'created_utc', 'sort': 'asc', 'size': 1000, 'subreddit': 'Pete_Buttigieg', 'type

{'sort_type': 'created_utc', 'sort': 'asc', 'size': 1000, 'subreddit': 'Pete_Buttigieg', 'type': 'comment', 'before': 1588291200, 'after': 1583332125}
{'sort_type': 'created_utc', 'sort': 'asc', 'size': 1000, 'subreddit': 'Pete_Buttigieg', 'type': 'comment', 'before': 1588291200, 'after': 1583333884}
{'sort_type': 'created_utc', 'sort': 'asc', 'size': 1000, 'subreddit': 'Pete_Buttigieg', 'type': 'comment', 'before': 1588291200, 'after': 1583335065}
{'sort_type': 'created_utc', 'sort': 'asc', 'size': 1000, 'subreddit': 'Pete_Buttigieg', 'type': 'comment', 'before': 1588291200, 'after': 1583335984}
{'sort_type': 'created_utc', 'sort': 'asc', 'size': 1000, 'subreddit': 'Pete_Buttigieg', 'type': 'comment', 'before': 1588291200, 'after': 1583337045}
{'sort_type': 'created_utc', 'sort': 'asc', 'size': 1000, 'subreddit': 'Pete_Buttigieg', 'type': 'comment', 'before': 1588291200, 'after': 1583338265}
{'sort_type': 'created_utc', 'sort': 'asc', 'size': 1000, 'subreddit': 'Pete_Buttigieg', 'type

{'sort_type': 'created_utc', 'sort': 'asc', 'size': 1000, 'subreddit': 'Pete_Buttigieg', 'type': 'comment', 'before': 1588291200, 'after': 1583418368}
{'sort_type': 'created_utc', 'sort': 'asc', 'size': 1000, 'subreddit': 'Pete_Buttigieg', 'type': 'comment', 'before': 1588291200, 'after': 1583419861}
{'sort_type': 'created_utc', 'sort': 'asc', 'size': 1000, 'subreddit': 'Pete_Buttigieg', 'type': 'comment', 'before': 1588291200, 'after': 1583421446}
{'sort_type': 'created_utc', 'sort': 'asc', 'size': 1000, 'subreddit': 'Pete_Buttigieg', 'type': 'comment', 'before': 1588291200, 'after': 1583422899}
pushshift failed
{'sort_type': 'created_utc', 'sort': 'asc', 'size': 1000, 'subreddit': 'Pete_Buttigieg', 'type': 'comment', 'before': 1588291200, 'after': 1583422899}
{'sort_type': 'created_utc', 'sort': 'asc', 'size': 1000, 'subreddit': 'Pete_Buttigieg', 'type': 'comment', 'before': 1588291200, 'after': 1583423948}
{'sort_type': 'created_utc', 'sort': 'asc', 'size': 1000, 'subreddit': 'Pete_

{'sort_type': 'created_utc', 'sort': 'asc', 'size': 1000, 'subreddit': 'Pete_Buttigieg', 'type': 'comment', 'before': 1588291200, 'after': 1583519916}
{'sort_type': 'created_utc', 'sort': 'asc', 'size': 1000, 'subreddit': 'Pete_Buttigieg', 'type': 'comment', 'before': 1588291200, 'after': 1583521378}
{'sort_type': 'created_utc', 'sort': 'asc', 'size': 1000, 'subreddit': 'Pete_Buttigieg', 'type': 'comment', 'before': 1588291200, 'after': 1583522764}
{'sort_type': 'created_utc', 'sort': 'asc', 'size': 1000, 'subreddit': 'Pete_Buttigieg', 'type': 'comment', 'before': 1588291200, 'after': 1583524435}
{'sort_type': 'created_utc', 'sort': 'asc', 'size': 1000, 'subreddit': 'Pete_Buttigieg', 'type': 'comment', 'before': 1588291200, 'after': 1583526070}
{'sort_type': 'created_utc', 'sort': 'asc', 'size': 1000, 'subreddit': 'Pete_Buttigieg', 'type': 'comment', 'before': 1588291200, 'after': 1583527823}
{'sort_type': 'created_utc', 'sort': 'asc', 'size': 1000, 'subreddit': 'Pete_Buttigieg', 'type

{'sort_type': 'created_utc', 'sort': 'asc', 'size': 1000, 'subreddit': 'Pete_Buttigieg', 'type': 'comment', 'before': 1588291200, 'after': 1583689908}
{'sort_type': 'created_utc', 'sort': 'asc', 'size': 1000, 'subreddit': 'Pete_Buttigieg', 'type': 'comment', 'before': 1588291200, 'after': 1583692323}
{'sort_type': 'created_utc', 'sort': 'asc', 'size': 1000, 'subreddit': 'Pete_Buttigieg', 'type': 'comment', 'before': 1588291200, 'after': 1583696353}
{'sort_type': 'created_utc', 'sort': 'asc', 'size': 1000, 'subreddit': 'Pete_Buttigieg', 'type': 'comment', 'before': 1588291200, 'after': 1583700095}
{'sort_type': 'created_utc', 'sort': 'asc', 'size': 1000, 'subreddit': 'Pete_Buttigieg', 'type': 'comment', 'before': 1588291200, 'after': 1583704024}
{'sort_type': 'created_utc', 'sort': 'asc', 'size': 1000, 'subreddit': 'Pete_Buttigieg', 'type': 'comment', 'before': 1588291200, 'after': 1583706779}
{'sort_type': 'created_utc', 'sort': 'asc', 'size': 1000, 'subreddit': 'Pete_Buttigieg', 'type